In [1]:
import json
f = open('json_file.json')
# get the json structure of the file f
data = json.load(f)

# print the json structure of the file f
d=data

In [1]:
from google.cloud import bigquery

# Initialiser le client BigQuery
client = bigquery.Client(project="vlille-396911")

In [89]:
# suppression de la table
client.delete_table('vlille-396911.vlille_dataset.vlille_table', not_found_ok=True)  # Make an API request.

In [90]:
# Définissez le schéma de la table
schema = [
    bigquery.SchemaField("recordid", "STRING"),
    bigquery.SchemaField("nbvelosdispo", "INT64"),
    bigquery.SchemaField("nbplacesdispo", "INT64"),
    bigquery.SchemaField("libelle", "INT64"),
    bigquery.SchemaField("adresse", "STRING"),
    bigquery.SchemaField("nom", "STRING"),
    bigquery.SchemaField("etat", "STRING"),
    bigquery.SchemaField("commune", "STRING"),
    bigquery.SchemaField("etatconnexion", "STRING"),
    bigquery.SchemaField("type", "STRING"),
    bigquery.SchemaField("longitude", "FLOAT64"),
    bigquery.SchemaField("latitude", "FLOAT64"),
    bigquery.SchemaField("datemiseajour", "TIMESTAMP"),
    bigquery.SchemaField("record_timestamp", "TIMESTAMP")
]

# Créez la table avec le schéma spécifié
table_id = "vlille-396911.vlille_dataset.vlille_table"
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)

print("Table créée avec succès:", table.table_id)


Table créée avec succès: vlille_table


In [69]:
!bq ls vlille_dataset

    tableId      Type    Labels   Time Partitioning   Clustered Fields  
 -------------- ------- -------- ------------------- ------------------ 
  allo           TABLE                                                  
  vlille_table   TABLE                                                  


In [81]:
data_to_insert = []

for record in data['records']:
    row = {
        "recordid": record['recordid'],
        "record_timestamp": record['record_timestamp'],
        "nbvelosdispo": record['fields']['nbvelosdispo'],
        "nbplacesdispo": record['fields']['nbplacesdispo'],
        "libelle": record['fields']['libelle'],
        "adresse": record['fields']['adresse'],
        "nom": record['fields']['nom'],
        "etat": record['fields']['etat'],
        "commune": record['fields']['commune'],
        "etatconnexion": record['fields']['etatconnexion'],
        "type": record['fields']['type'],
        "longitude": record['fields']['localisation'][0],
        "latitude": record['fields']['localisation'][1],
        "datemiseajour": record['fields']['datemiseajour']
    }

    data_to_insert.append(row)

In [2]:
dataset_id = 'vlille_dataset'
table_id = 'vlille_table'
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)  # API call

In [73]:
errors = client.insert_rows(table, data_to_insert)  # API request
if errors == []:
    print("Data inserted successfully.")
else:
    print("Encountered errors while inserting rows:", errors)

Data inserted successfully.


In [88]:
client.insert_rows(table, data_to_insert)

[]

In [3]:
query = f'SELECT * FROM `{dataset_id}.{table_id}` ORDER BY `datemiseajour`'
query_job = client.query(query)  # API request
results = query_job.result()  # Waits for query to finish
# for row in results:
#     print(row)
print(results.total_rows)

132940
